In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

def main():
    # --------------------------------------------------------------------------
    # 1. READ CSV FILE (ADAPT PATH AND COLUMN NAMES AS NEEDED)
    # --------------------------------------------------------------------------
    # Suppose the CSV has columns like: AgeDelta, Sex, Diabetes, VAT, ASAT, MATI, TherapyGroup
    df = pd.read_csv("path_to_your_data.csv")  # Replace with your actual CSV file path

    # Convert TherapyGroup to a categorical if it isn't already
    df["TherapyGroup"] = df["TherapyGroup"].astype("category")

    # Example: Convert Sex to 0/1 if it is "M"/"F" (adjust if needed)
    # df["Sex"] = df["Sex"].map({"M": 0, "F": 1})

    # --------------------------------------------------------------------------
    # 2. SPECIFY AND FIT YOUR MODEL
    # --------------------------------------------------------------------------
    # This formula demonstrates interactions similar to the table you showed.
    # Replace 'AgeDelta' with your actual outcome if it differs.
    formula = """
        AgeDelta 
        ~ Sex 
        + Diabetes 
        + VAT:Diabetes 
        + ASAT:Diabetes 
        + MATI:Diabetes
        + VAT:Diabetes:TherapyGroup 
        + ASAT:Diabetes:TherapyGroup 
        + MATI:Diabetes:TherapyGroup
    """
    model = smf.ols(formula, data=df).fit()

    # --------------------------------------------------------------------------
    # 3. EXTRACT RELEVANT STATISTICS (COEFS, STD ERR, T, P, CONF. INTERVALS)
    # --------------------------------------------------------------------------
    # Collect model parameters in a structured format
    params = model.params
    conf_intervals = model.conf_int()
    pvalues = model.pvalues
    stderr = model.bse
    tvals = model.tvalues

    # Create a list of dictionaries for easier printing or further manipulation
    results_rows = []
    for param in params.index:
        ci_low, ci_high = conf_intervals.loc[param]
        results_rows.append({
            "Variable": param if param != "Intercept" else "Intercept",
            "coef": params[param],
            "std_err": stderr[param],
            "t_value": tvals[param],
            "p_value": pvalues[param],
            "CI_low": ci_low,
            "CI_high": ci_high
        })

    # --------------------------------------------------------------------------
    # 4. PRINT OUT THE RESULTS IN A SIMPLE TEXT FORMAT
    # --------------------------------------------------------------------------
    # This is just a demonstration. You could write them to another CSV, etc.
    print("Variable, coef, std_err, t_value, p_value, CI_low, CI_high")
    for row in results_rows:
        print(f"{row['Variable']}, "
              f"{row['coef']:.4f}, "
              f"{row['std_err']:.4f}, "
              f"{row['t_value']:.4f}, "
              f"{row['p_value']:.4f}, "
              f"({row['CI_low']:.4f}, {row['CI_high']:.4f})")

    # Optional: Print the model summary if you want a full readout
    # print(model.summary())

if __name__ == "__main__":
    main()